In [8]:
%load_ext autoreload
%autoreload 2
import os

from sentence_transformers import SentenceTransformer
from src.text_embedder.embedder import text_to_embedding_transformer

import pandas as pd
import torch
from loguru import logger
import pickle

from src.text_embedder.embedder import EmbeddingsModel

import tqdm
from pathlib import Path
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Data

In [9]:
# Load csv file
# get current path

notebook_path = Path().resolve()
print(notebook_path)

image_path = os.path.join(
    notebook_path,
    "data",
    "raw",
    "biology_chapter_3_3_treshhold_5",
    "extracted_keyframes",
)


filename = "experiments/first_experiment/test_frames_gt_new.csv"
df = pd.read_csv(filename)
df

C:\Users\baatout\PycharmProjects\afm-vlm\experiments\second_experiment\notebooks


FileNotFoundError: [Errno 2] No such file or directory: 'experiments/first_experiment/test_frames_gt_new.csv'

In [ ]:
# print pickle file 
with open("experiments/first_experiment/bio_3_3_th5.pickle", "rb") as file:
    data = pickle.load(file)

logger.info(f"new_pickle: {data}")

In [76]:

# set image paths for the experiments

embedding_model = EmbeddingsModel()

extracted_data_path = [data[key]['img_path'] for key in data.keys() if
                       'img_path' in data[key]]

embedding_model.img_paths = None

embedding_model.img_paths = extracted_data_path

extracted_data_ocr_text = [data[key]['ocr_extracted_text'] for key in data.keys() if
                           'ocr_extracted_text' in data[key]]

logger.info(f"Extracted_data_ocr_text: {extracted_data_ocr_text}")

logger.info(f"extracted_data_path: {extracted_data_path}")

# Note: the data is correctly mapped 
print(extracted_data_path[0])
print(extracted_data_ocr_text[0])

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: paraphrase-multilingual-MiniLM-L12-v2
2024-07-03 19:18:31.495 | INFO     | __main__:<module>:15 - Extracted_data_ocr_text: ['Transport Across Membranes\n\n+ Exocytosis: a membrane-bound vesicle fuses with the\nmembrane and expels the large molecule outside the cell\n\n(2) Exocytosis\n\n', 'Do you use any of these supplements?\n\n>\n\ngingko\nkava\nginseng\n\nmelatonin\n\nmooo\n\ncoconut water\n\n‘$0 YOU WAVE VITAMINS, AND THEN YOU HAVE "HERBAL\nSUPPLEMENTS." WHICH ARE WOT FDA APPROVED.\n\nL\n‘AND JUST FOR REFERENCE, THE TWINKIE IS.\n\n', 'Transport Across Membranes\n\n* Membrane transport\n— Plasma membrane:\nphospholipid bilayer is\ndifferentially permeable\n= Hydrophobic substances\n\n', 'Nutrients: Macronutrients\n\n* Carbohydrates: main energy\n\nsource Fructose e\n+ Possible structural composition: =\n— Single-unit simple s

/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3_treshhold_5/extracted_keyframes/biology_chapter_3_3_treshhold_5-Scene-079-01.jpg
Transport Across Membranes

+ Exocytosis: a membrane-bound vesicle fuses with the
membrane and expels the large molecule outside the cell

(2) Exocytosis




<Figure size 800x2000 with 0 Axes>

In [ ]:
# create dataframe to save results of the experiments
columns = ['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3']

df_test = pd.DataFrame(columns=columns)

df_ocr_only = df_test
df_ocr_lava = df_test
df_ocr_transcriptions = df_test

df_short_llm_summary = df_test
df_extensive_summary = df_test

df_clip_llm_summary = df_test
df_clip_extensive_summary = df_test

In [ ]:
def add_to_df(prompt, gt_keyframe, result):
    # Create a new row with the provided data
    return {
        'Prompt': prompt,
        'GT_Keyframe': gt_keyframe,
        'Top_1': extract_keyframe_number(result[0]) if len(result) > 0 else None,
        'Top_2': extract_keyframe_number(result[1]) if len(result) > 1 else None,
        'Top_3': extract_keyframe_number(result[2]) if len(result) > 2 else None
    }


# ietrate over the dataframe and get the results
def get_results(df):
    for _, row in df.iterrows():
        logger.info(row['Prompt'])
        prompt = row['Prompt']
        gt_keyframe = row['GT_Keyframe']

        # Search for similar images
        output = embedding_model.search_similar_images_top_k(prompt, gt_keyframe, 3)
        res_row = add_to_df(prompt, gt_keyframe, output)
        rows.append(res_row)

    return rows


def extract_keyframe_number(path):
    """
    Extracts the scene number from the given file path.

    Parameters:
    path (str): The full path of the file.

    Returns:
    str: The extracted scene number.
    """
    # Get filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]

    # Extract '032' from filename
    scene_number = filename.split('-Scene-')[-1].split('-')[0]

    return scene_number

In [80]:
# Embedded with standard Tokenizer: Only OCR
embedding_model.text_embeddings = None

logger.info(f"Embedded with standard Tokenizer: Only OCR")

extracted_data_ocr_text = [data[key]['ocr_extracted_text'] for key in data.keys() if
                           'ocr_extracted_text' in data[key]]

#logger.info(extracted_data_ocr_text)

# get the embedder model
embedder_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

ocr_embeddings = text_to_embedding_transformer(extracted_data_ocr_text, embedder_model)

#logger.info('OCR Embeddings: ', ocr_embeddings)

embedding_model.text_embeddings = ocr_embeddings

rows = []

rows = get_results(df)

df_ocr_only = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_ocr_only.to_csv('df_ocr_only.csv', index=False)

2024-07-03 19:19:00.426 | INFO     | __main__:<module>:4 - Embedded with standard Tokenizer: Only OCR
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: paraphrase-multilingual-MiniLM-L12-v2


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2024-07-03 19:19:06.585 | INFO     | __main__:get_results:15 - learning outcomes


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-07-03 19:19:06.646 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:167 - Query text embedding shape: torch.Size([384])
2024-07-03 19:19:06.646 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:168 - Text embeddings shape: torch.Size([85, 384])
2024-07-03 19:19:06.650 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:175 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.3722, 0.2852, 0.2718]),
indices=tensor([ 8, 63, 62])) - GT is keyframe number 3
2024-07-03 19:19:06.650 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:177 - Length of img paths: 85
2024-07-03 19:19:06.654 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:185 - #####GT is keyframe number 3#####
2024-07-03 19:19:06.655 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:186 - Max similarity for index 8 is the keyframe /Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3_tr

...

2024-07-03 19:19:07.905 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:167 - Query text embedding shape: torch.Size([384])
2024-07-03 19:19:07.905 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:168 - Text embeddings shape: torch.Size([85, 384])
2024-07-03 19:19:07.909 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:175 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.7310, 0.7236, 0.7230]),
indices=tensor([75, 12, 57])) - GT is keyframe number 70
2024-07-03 19:19:07.909 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:177 - Length of img paths: 85
2024-07-03 19:19:07.913 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:185 - #####GT is keyframe number 70#####
2024-07-03 19:19:07.913 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:186 - Max similarity for index 75 is the keyframe /Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-07-03 19:19:07.987 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:167 - Query text embedding shape: torch.Size([384])
2024-07-03 19:19:07.987 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:168 - Text embeddings shape: torch.Size([85, 384])
2024-07-03 19:19:07.990 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:175 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.4662, 0.2444, 0.2037]),
indices=tensor([10,  1, 38])) - GT is keyframe number 84
2024-07-03 19:19:07.990 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:177 - Length of img paths: 85
2024-07-03 19:19:07.993 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:185 - #####GT is keyframe number 84#####
2024-07-03 19:19:07.994 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:186 - Max similarity for index 10 is the keyframe /Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-07-03 19:19:08.071 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:167 - Query text embedding shape: torch.Size([384])
2024-07-03 19:19:08.073 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:168 - Text embeddings shape: torch.Size([85, 384])
2024-07-03 19:19:08.075 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:175 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.2261, 0.2256, 0.2142]),
indices=tensor([16, 10, 76])) - GT is keyframe number 84
2024-07-03 19:19:08.075 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:177 - Length of img paths: 85
2024-07-03 19:19:08.075 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:185 - #####GT is keyframe number 84#####
2024-07-03 19:19:08.075 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:186 - Max similarity for index 16 is the keyframe /Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3

In [22]:
top_3 = []
embedding_model.check_proximity_keyframes(90, top_3)

[84, 83, 82, 81, 80]

# LOAD MODEL 

In [ ]:
# Embedded with standard Tokenizer: OCR * Transcriptions 
logger.info(f"Embedded with standard Tokenizer: OCR * Transcriptions")

embedding_model.text_embeddings = None

extracted_data_ocr_text = [data[key]['ocr_extracted_text'] for key in data.keys() if
                           'ocr_extracted_text' in data[key]]

extracted_data_transcriptions = [data[key]['transcription'] for key in data.keys() if
                                 'transcription' in data[key]]

# concatenate ocr and transcriptions
concat_result = [a + ' ' + b for a, b in zip(extracted_data_ocr_text, extracted_data_transcriptions)]

# get the embedder model
embedder_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

concat_result_embeddings = text_to_embedding_transformer(concat_result, embedder_model)

embedding_model.text_embeddings = concat_result_embeddings

result = []

rows = get_results(df)

df_ocr_transcriptions = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# save on disk
df_ocr_transcriptions.to_csv('df_ocr_transcriptions.csv', index=False)

In [ ]:
# Embedded with standard Tokenizer: OCR * LLAVA
logger.info(f"Embedded with standard Tokenizer: OCR * LLAVA")

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = extracted_data_path

extracted_data_ocr_text = [data[key]['ocr_extracted_text'] for key in data.keys() if
                           'ocr_extracted_text' in data[key]]

extracted_data_llava_result = [data[key]['llava_result'] for key in data.keys() if
                               'llava_result' in data[key]]

# concatenate ocr and transcriptions
concat_result = [a + ' ' + b for a, b in zip(extracted_data_ocr_text, extracted_data_llava_result)]

# get the embedder model
embedder_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

concat_result_embeddings = text_to_embedding_transformer(concat_result, embedder_model)

embedding_model.text_embeddings = concat_result_embeddings

result = []

rows = get_results(df)

df_ocr_lava = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# save on disk
df_ocr_lava.to_csv('df_ocr_lava.csv', index=False)

# Load new data from CSV

In [7]:
# load data from csv

df_data_extensive = pd.read_csv('experiments/second_experiment/data_generation_pipeline_010724.csv')
df_data_extensive

FileNotFoundError: [Errno 2] No such file or directory: 'experiments/second_experiment/data_generation_pipeline_010724.csv'

In [ ]:
# get llm_long_summary 
llm_long_summary = df_data_extensive.iloc[6].to_dict().values()

# convert to list
llm_long_summary = list(llm_long_summary)

In [ ]:
# get ocr_only
ocr_only = df_data_extensive.iloc[11].to_dict().values()

# convert to list
ocr_only = list(ocr_only)

In [ ]:
# get ocr_lava ----- NOT EXISTING
ocr_lava = df_data_extensive.iloc[6].to_dict().values()

# convert to list
ocr_lava = list(ocr_lava)

In [ ]:
# get ocr_transcriptions
ocr_transcriptions = df_data_extensive.iloc[14].to_dict().values()

# convert to list
ocr_transcriptions = list(ocr_transcriptions)

In [51]:
# get img_path
img_path =  df_data_extensive.iloc[0].to_dict().values()

# convert to list
img_path = list(img_path)
img_path

['img_path',
 '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3_treshhold_5/extracted_keyframes/biology_chapter_3_3_treshhold_5-Scene-079-01.jpg',
 '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3_treshhold_5/extracted_keyframes/biology_chapter_3_3_treshhold_5-Scene-004-01.jpg',
 '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3_treshhold_5/extracted_keyframes/biology_chapter_3_3_treshhold_5-Scene-067-01.jpg',
 '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3_treshhold_5/extracted_keyframes/biology_chapter_3_3_treshhold_5-Scene-016-01.jpg',
 '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3_treshhold_5/extracted_keyframes/biology_chapter_3_3_treshhold_5-Scene-075-01.jpg',
 '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3_treshhold_5/extracted_keyframes/biology_chapter_3_3_treshhold_5-Scene-008-01.jpg',
 '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/ra

In [81]:
# Embedd with LLM long summary
logger.info(f"Embedd with LLM long summary")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path

embeddings = embedding_model.generate_dataset_embeddings_standard_tokenizer(llm_long_summary)

embedding_model.text_embeddings = embeddings

logger.info(f"Embedding Model.text_embeddings: {embedding_model.text_embeddings}")

rows = []

rows = get_results(df)

df_extensive_summary = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_extensive_summary.to_csv('df_standard_llm_long_summary.csv', index=False)


2024-07-03 19:19:33.395 | INFO     | __main__:<module>:2 - Embedd with LLM long summary
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: paraphrase-multilingual-MiniLM-L12-v2


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2024-07-03 19:19:43.078 | INFO     | __main__:<module>:18 - Embedding Model.text_embeddings: tensor([[-0.1943,  0.0935,  0.1131,  ..., -0.0911, -0.2710,  0.1646],
        [ 0.0253, -0.0263, -0.0014,  ...,  0.4360,  0.3577,  0.1273],
        [-0.3051, -0.3519, -0.0007,  ...,  0.1595,  0.0330,  0.1471],
        ...,
        [-0.0642, -0.2373,  0.4589,  ..., -0.1966,  0.1491, -0.1945],
        [-0.1577, -0.1107, -0.1429,  ...,  0.0462,  0.2383, -0.0868],
        [-0.2274, -0.1598,  0.0243,  ..., -0.0246,  0.0617, -0.0040]])
2024-07-03 19:19:43.078 | INFO     | __main__:get_results:15 - learning outcomes


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-07-03 19:19:43.135 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:167 - Query text embedding shape: torch.Size([384])
2024-07-03 19:19:43.135 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:168 - Text embeddings shape: torch.Size([86, 384])
2024-07-03 19:19:43.145 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:175 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.4803, 0.4311, 0.3226]),
indices=tensor([64, 63, 79])) - GT is keyframe number 3
2024-07-03 19:19:43.145 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:177 - Length of img paths: 86
2024-07-03 19:19:43.148 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:185 - #####GT is keyframe number 3#####
2024-07-03 19:19:43.149 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:186 - Max similarity for index 64 is the keyframe /Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3_t

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-07-03 19:19:44.496 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:167 - Query text embedding shape: torch.Size([384])
2024-07-03 19:19:44.496 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:168 - Text embeddings shape: torch.Size([86, 384])
2024-07-03 19:19:44.500 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:175 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.5299, 0.3209, 0.2511]),
indices=tensor([11, 39, 42])) - GT is keyframe number 84
2024-07-03 19:19:44.500 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:177 - Length of img paths: 86
2024-07-03 19:19:44.504 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:185 - #####GT is keyframe number 84#####
2024-07-03 19:19:44.505 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:186 - Max similarity for index 11 is the keyframe /Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-07-03 19:19:44.570 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:167 - Query text embedding shape: torch.Size([384])
2024-07-03 19:19:44.573 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:168 - Text embeddings shape: torch.Size([86, 384])
2024-07-03 19:19:44.576 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:175 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.2838, 0.2779, 0.2626]),
indices=tensor([11, 71, 38])) - GT is keyframe number 84
2024-07-03 19:19:44.576 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:177 - Length of img paths: 86
2024-07-03 19:19:44.576 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:185 - #####GT is keyframe number 84#####
2024-07-03 19:19:44.576 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:186 - Max similarity for index 11 is the keyframe /Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3

<Figure size 800x2000 with 0 Axes>